# **사전 준비**

In [ ]:
!pip install langchain langchain_openai langchain_community pypdf faiss-cpu

In [ ]:
from google.colab import drive
import os

# 먼저 구글 드라이브 마운트
drive.mount('/content/drive')

In [ ]:

import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")

# 환경 변수에서 API 키 가져오기
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = (
    "파일 경로"
)
loader = PyPDFLoader(file_path)

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
docs = loader.load_and_split(doc_splitter)